<a href="https://colab.research.google.com/github/LucasAlegre/vote-network/blob/master/vote_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install and import libraries

In [1]:
! pip install --user graphistry
! pip install python-igraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.7 MB/s eta 0:00:00


In [1]:
import graphistry
import pandas as pd
import urllib.request
import requests
from datetime import date
import numpy as np
from igraph import Graph, summary
from itertools import combinations
import os
# Init graphistry
graphistry.register(api=3, protocol="https", server="hub.graphistry.com", username="LucasAlegre", password="")

/usr/local/lib/python3.10/dist-packages/cudf/utils/_ptxcompiler.py:64: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/driver.py", line 295, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/cudf/utils/gpu_utils.py:62: UserWarning: Failed to dlopen libcuda.so.1
  warnings.warn(str(e))


## Data Retrieval and Processing

In [2]:
start_date = '2024-01-01' #@param {type:"date"}
end_date = '2024-12-31' #@param {type:"date"}
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

In [3]:
# Collect all votes between start and end dates
os.makedirs('data', exist_ok=True)
for year in range(start_date.year, end_date.year + 1):
    # Read or download votes
    path = 'data/votacoesVotos-{}.csv'.format(year)
    if not os.path.isfile(path):
      headers = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'}
      url = 'https://dadosabertos.camara.leg.br/arquivos/votacoesVotos/csv/votacoesVotos-{}.csv'.format(year)
      request = urllib.request.Request(url, None, headers) #The assembled request
      response = urllib.request.urlopen(request)
      data = response.read() # The data u need
      with open(path, 'wb') as f:
          f.write(data)

    vote_data = pd.read_csv(path, sep=';')

    # Remove simbolic votes
    vote_data = vote_data[vote_data['voto'] != 'Simbólico']

    # Filter dates
    vote_data['dataHoraVoto'] = pd.to_datetime(vote_data['dataHoraVoto'])
    vote_data = vote_data.loc[(vote_data['dataHoraVoto'] >= start_date) & (vote_data['dataHoraVoto'] <= end_date)]
    vote_data.sort_values(by=['dataHoraVoto'], inplace=True)

    if year == start_date.year:
        votes = vote_data
    else:
        votes = pd.concat([votes, vote_data])
votes

,idVotacao,uriVotacao,dataHoraVoto,voto,deputado_id,deputado_uri,deputado_nome,deputado_siglaPartido,deputado_uriPartido,deputado_siglaUf,deputado_idLegislatura,deputado_urlFoto
26698,2367548-7,https://dadosabertos.camara.leg.br/api/v2/vota...,2024-02-20 20:48:55,Artigo 17,160541,https://dadosabertos.camara.leg.br/api/v2/depu...,Arthur Lira,PP,https://dadosabertos.camara.leg.br/api/v2/part...,AL,57,https://www.camara.leg.br/internet/deputado/ba...
26853,2367548-7,https://dadosabertos.camara.leg.br/api/v2/vota...,2024-02-20 20:49:10,Sim,198197,https://dadosabertos.camara.leg.br/api/v2/depu...,Greyce Elias,AVANTE,https://dadosabertos.camara.leg.br/api/v2/part...,MG,57,https://www.camara.leg.br/internet/deputado/ba...
27078,2367548-7,https://dadosabertos.camara.leg.br/api/v2/vota...,2024-02-20 20:49:10,Sim,220598,https://dadosabertos.camara.leg.br/api/v2/depu...,Tarcísio Motta,PSOL,https://dadosabertos.camara.leg.br/api/v2/part...,RJ,57,https://www.camara.leg.br/internet/deputado/ba...
26843,2367548-7,https://dadosabertos.camara.leg.br/api/v2/vota...,2024-02-20 20:49:11,Sim,204491,https://dadosabertos.camara.leg.br/api/v2/depu...,Gilberto Abramo,REPUBLICANOS,https://dadosabertos.camara.leg.br/api/v2/part...,MG,57,https://www.camara.leg.br/internet/deputado/ba...
26903,2367548-7,https://dadosabertos.camara.leg.br/api/v2/vota...,2024-02-20 20:49:12,Sim,204457,https://dadosabertos.camara.leg.br/api/v2/depu...,Juninho do Pneu,UNIÃO,https://dadosabertos.camara.leg.br/api/v2/part...,RJ,57,https://www.camara.leg.br/internet/deputado/ba...
...,...,...,...,...,...,...,...,...,...,...,...,...
14310,2315917-49,https://dadosabertos.camara.leg.br/api/v2/vota...,2024-10-29 18:56:25,Sim,220654,https://dadosabertos.camara.leg.br/api/v2/depu...,Delegado Paulo Bilynskyj,PL,https://dadosabertos.camara.leg.br/api/v2/part...,SP,57,https://www.camara.leg.br/internet/deputado/ba...
14309,2315917-49,https://dadosabertos.camara.leg.br/api/v2/vota...,2024-10-29 18:56:25,Sim,204512,https://dadosabertos.camara.leg.br/api/v2/depu...,Delegado Marcelo Freitas,UNIÃO,https://dadosabertos.camara.leg.br/api/v2/part...,MG,57,https://www.camara.leg.br/internet/deputado/ba...
14308,2315917-49,https://dadosabertos.camara.leg.br/api/v2/vota...,2024-10-29 18:56:25,Sim,220539,https://dadosabertos.camara.leg.br/api/v2/depu...,Defensor Stélio Dener,REPUBLICANOS,https://dadosabertos.camara.leg.br/api/v2/part...,RR,57,https://www.camara.leg.br/internet/deputado/ba...
14306,2315917-49,https://dadosabertos.camara.leg.br/api/v2/vota...,2024-10-29 18:56:25,Sim,220594,https://dadosabertos.camara.leg.br/api/v2/depu...,Coronel Assis,UNIÃO,https://dadosabertos.camara.leg.br/api/v2/part...,MT,57,https://www.camara.leg.br/internet/deputado/ba...


In [4]:
def get_total_expenses(deputy_id, start_date, end_date):
    #expenses = []
    total = 0.0
    years = '&'.join(['ano={}'.format(year) for year in range(start_date.year, end_date.year+1)])
    url = "https://dadosabertos.camara.leg.br/api/v2/deputados/{}/despesas?{}&itens=100000&ordem=ASC&ordenarPor=ano".format(deputy_id, years)
    read_all = False
    while not read_all:
        read_all = True
        page = requests.get(url).json()
        if 'dados' not in page:
            break
        total += sum([e['valorDocumento'] for e in page['dados']])
        #this_expenses = [{'tipo': e['tipoDespesa'], 'valor': e['valorDocumento']} for e in page['dados']]
        #expenses.extend(this_expenses)
        for link in page['links']:
            if link['rel'] == 'next':
                url = link['href']
                read_all = False
                break
    return total

def get_deputy_info(deputy_id):
    url = 'https://dadosabertos.camara.leg.br/api/v2/deputados/{}'.format(deputy_id)
    data = requests.get(url).json()['dados']
    ultimoStatus = data['ultimoStatus']
    today = date.today()
    nascimento = pd.to_datetime(data['dataNascimento'])
    idade = today.year - nascimento.year - ((today.month, today.day) < (nascimento.month, nascimento.day))
    return {'sexo': data['sexo'],
            'escolaridade': data['escolaridade'],
            'idade': idade,
            'e-mail': ultimoStatus['email'],
            'situação': ultimoStatus['situacao'],
            'cidade natal': data['municipioNascimento']}

In [5]:
#%% Take care of different names for same deputy
for group, df_group in votes.groupby('deputado_id'):
    votes['deputado_nome'].loc[votes['deputado_id'] == group] = sorted(df_group['deputado_nome'].unique())[0]

#%% Partidos que mudaram de nome
votes['deputado_siglaPartido'].replace('PMDB', 'MDB', inplace=True)
votes['deputado_siglaPartido'].replace('PRB', 'REPUBLICANOS', inplace=True)
votes['deputado_siglaPartido'].replace('PR', 'PL', inplace=True)
votes['deputado_siglaPartido'].replace('PATRIOTA', 'PATRI', inplace=True)
votes['deputado_siglaPartido'].replace('PPS', 'CIDADANIA', inplace=True)

# Logo dos partidos
party_logo = {}
for p in votes['deputado_uriPartido'].unique():
    if not pd.isna(p):
        dados = requests.get(p).json()['dados']
        if dados['sigla'] == 'MDB':
            party_logo[p] = 'https://logodownload.org/wp-content/uploads/2018/04/mdb-logo-partido.png'
        elif dados['sigla'] == 'SOLIDARIEDADE':
            party_logo[p] = 'https://upload.wikimedia.org/wikipedia/commons/f/fe/Logomarca_do_Partido_Solidariedade.png'
        elif dados['sigla'] == 'PATRI':
            party_logo[p] = 'https://upload.wikimedia.org/wikipedia/en/8/8e/Patriota_logo.png'
        elif dados['sigla'] == 'REPUBLICANOS':
            party_logo[p] = 'https://upload.wikimedia.org/wikipedia/en/0/0d/Republicanos_logo.png'
        elif dados['sigla'] == 'PL':
            party_logo[p] = 'https://upload.wikimedia.org/wikipedia/commons/0/03/PL-logo.jpg'
        elif dados['sigla'] == 'CIDADANIA':
            party_logo[p] = 'https://upload.wikimedia.org/wikipedia/commons/d/d7/Logo_do_Cidadania_23.png'
        elif dados['sigla'] == 'NOVO':
            party_logo[p] = 'https://upload.wikimedia.org/wikipedia/commons/b/b5/Novo30_AOC.png'
        elif dados['sigla'] == 'UNIÃO':
            party_logo[p] = 'https://upload.wikimedia.org/wikipedia/commons/7/73/Uni%C3%A3o_Brasil_logo.svg'
        else:
            party_logo[p] = dados['urlLogo']

#all_data['deputado_siglaPartido'].replace('PPL', np.nan, inplace=True) # PPL foi incorporado
#all_data['deputado_siglaPartido'].replace('PRP', np.nan, inplace=True) # PRP foi incorporado
#all_data['deputado_siglaPartido'].replace('PHS', np.nan, inplace=True) # PHS foi incorporado
# all_data = pd.merge(all_data, motions_themes, on="idVotacao", how="inner")
#all_data['deputado_siglaPartido'].fillna('S.PART.', inplace=True)
# all_data.groupby('idVotacao')['voto'].count()

#all_data.to_csv('votos_{}_to_{}.csv'.format(start_date, end_date), index=False)

Streaming output truncated to the last 5000 lines.
<ipython-input-5-c7db012ac8e2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  votes['deputado_nome'].loc[votes['deputado_id'] == group] = sorted(df_group['deputado_nome'].unique())[0]
<ipython-input-5-c7db012ac8e2>:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer,

## Graph Construction

In [6]:
def generalized_similarity(m, min_eps=0.001, max_iter=1000):
    """ Balázs Kovács, "A generalized model of relational similarity," Social Networks, 32(3), July 2010, pp. 197–211
        Based on: https://github.com/dzinoviev/generalizedsimilarity
    """
    arcs0 = m - m.mean(axis=1)[:, np.newaxis]
    arcs1 = m.T - m.mean(axis=0)[:, np.newaxis]

    eps = min_eps + 1
    N = np.eye(m.shape[1])

    iters = 0
    while (eps > min_eps and iters < max_iter) or np.isnan(N).any():
        M = arcs0.dot(N).dot(arcs0.T)
        m = np.sqrt(M.diagonal())
        M = ((M / (m+1e-8)).T / (m+1e-8)).T

        Np = arcs1.dot(M).dot(arcs1.T)
        n = np.sqrt(Np.diagonal())
        Np = ((Np / (n+1e-8)).T / (n+1e-8)).T
        eps = np.abs(Np - N).max()
        N = Np

        iters += 1
    return M

def pearson_correlation(m):
    arcs0 = m - m.mean(axis=1)[:, np.newaxis]
    arcs1 = m.T - m.mean(axis=0)[:, np.newaxis]
    M = arcs0.dot(arcs0.T)
    m = np.sqrt(M.diagonal())
    M = ((M / m).T / m).T
    return M

def filter_edges(edges_list, num_nodes, threshold=None, density=0.1):
    edges, weights = [], []
    if threshold is not None:
        for e in edges_list:
            if e[1] >= threshold:
                edges.append(e[0])
                weights.append(e[1])
    else:
        count = int(num_nodes * (num_nodes - 1) * density / 2)
        edges_list.sort(reverse=True, key=lambda e: e[1])
        edges_list = edges_list[:count]
        edges = [e[0] for e in edges_list]
        weights = [e[1] for e in edges_list]
    return edges, weights

In [7]:
# Vote Matrix
reps = votes['deputado_nome'].unique()
rep_to_ind = {reps[i]: i for i in range(len(reps))}
motions = votes['idVotacao'].unique()
motion_to_ind = {motions[i]: i for i in range(len(motions))}
parties = [p for p in votes['deputado_siglaPartido'].unique() if pd.notna(p)]

vote_matrix = np.zeros((len(reps), len(motions)))
df_grouped = votes.groupby(['idVotacao', 'deputado_nome'])
for group, df_group in df_grouped:
    voto = df_group['voto'].values[0]
    i = rep_to_ind[group[1]]
    j = motion_to_ind[group[0]]
    if voto == "Sim":
        vote_matrix[i,j] = 1
    elif voto == "Não":
        vote_matrix[i,j] = -1

In [8]:
# Edges Generation
M = generalized_similarity(vote_matrix)
edges = []
for dep1, dep2 in combinations(range(len(reps)), 2):
    if M[dep1,dep2] > 0:
        edges.append(((dep1,dep2), M[dep1,dep2]))

In [9]:
# i-graph
graph = Graph(graph_attrs={'name': 'Câmara dos Deputados'}, directed=False)
graph.add_vertices(reps)
filer_edges, weights = filter_edges(edges, num_nodes=graph.vcount(), threshold=None, density=0.2) # threshold 0.9998
graph.add_edges(filer_edges)
graph.es['weight'] = weights
graph.es['similarity'] = weights
maxw = max(graph.es['weight'])
minw = min(graph.es['weight'])
graph.es['weight'] = [(e - minw) / (maxw - minw) for e in graph.es['weight']] # Normalize weights to [0,1]

In [10]:
# Community Detection
graph.vs['community'] = graph.community_leiden(weights='weight', objective_function='modularity', resolution_parameter=1, n_iterations=100).membership
graph.vs['community_leiden'] = [str(c) for c in graph.vs['community']]
print(graph.vs['community'])
print(graph.modularity(graph.vs['community'], weights='weight'))
""" graph.delete_edges()
edges, weights = filter_edges(edges, num_nodes=graph.vcount(), threshold=0.0, density=0.1)
graph.add_edges(edges)
graph.es['weight'] = weights
graph.es['similarity'] = weights """

[0, 1, 0, 2, 2, 2, 2, 3, 4, 0, 0, 3, 2, 2, 0, 0, 5, 2, 0, 6, 4, 4, 0, 0, 4, 2, 6, 3, 2, 4, 2, 4, 6, 6, 2, 0, 0, 2, 2, 0, 7, 6, 2, 4, 0, 3, 2, 2, 4, 2, 0, 8, 0, 2, 8, 0, 2, 2, 4, 0, 0, 2, 0, 2, 4, 4, 0, 0, 9, 3, 0, 2, 2, 4, 4, 0, 0, 0, 3, 0, 4, 4, 0, 4, 2, 2, 0, 0, 0, 6, 4, 4, 0, 6, 0, 3, 8, 4, 0, 2, 4, 4, 2, 4, 0, 2, 3, 0, 2, 4, 8, 8, 10, 2, 4, 4, 2, 8, 2, 8, 2, 0, 4, 0, 2, 11, 4, 0, 2, 2, 2, 8, 8, 2, 0, 4, 4, 0, 4, 2, 4, 2, 2, 4, 4, 4, 0, 6, 2, 0, 4, 2, 0, 8, 4, 2, 2, 2, 4, 2, 0, 0, 2, 0, 2, 4, 12, 2, 0, 2, 2, 0, 3, 13, 4, 0, 4, 2, 4, 0, 0, 0, 14, 0, 2, 0, 4, 2, 2, 15, 2, 2, 2, 0, 0, 3, 0, 0, 4, 2, 4, 4, 0, 4, 2, 0, 0, 2, 4, 2, 0, 0, 4, 4, 4, 4, 0, 4, 4, 0, 2, 4, 4, 0, 4, 6, 4, 4, 4, 2, 0, 4, 2, 2, 2, 0, 4, 4, 0, 0, 0, 4, 0, 6, 0, 2, 3, 0, 0, 0, 0, 4, 4, 4, 12, 0, 0, 2, 0, 0, 4, 4, 4, 0, 0, 2, 2, 4, 0, 2, 4, 0, 4, 16, 2, 4, 2, 2, 2, 0, 0, 2, 4, 4, 8, 3, 6, 0, 4, 2, 4, 2, 2, 2, 4, 4, 2, 0, 2, 4, 12, 3, 0, 2, 0, 0, 0, 2, 0, 0, 2, 2, 2, 2, 6, 4, 0, 0, 0, 2, 4, 0, 4, 4, 2, 2, 8, 2, 13, 4,

<ipython-input-10-9160391992dc>:2: DeprecationWarning: resolution_parameter keyword argument is deprecated, use resolution=... instead
  graph.vs['community'] = graph.community_leiden(weights='weight', objective_function='modularity', resolution_parameter=1, n_iterations=100).membership


" graph.delete_edges()\nedges, weights = filter_edges(edges, num_nodes=graph.vcount(), threshold=0.0, density=0.1)\ngraph.add_edges(edges)\ngraph.es['weight'] = weights\ngraph.es['similarity'] = weights "

In [11]:
# Add info to nodes
info = [votes[votes['deputado_nome']==dep] for dep in graph.vs['name']]
graph.vs['Foto'] = [x['deputado_urlFoto'].values[-1] for x in info]
graph.vs['UF'] = [x['deputado_siglaUf'].values[-1] for x in info]
graph.vs['Partido'] = [x['deputado_siglaPartido'].values[-1] for x in info]
graph.vs['URL'] =  [x['deputado_uri'].values[-1] for x in info]
graph.vs['Partido URI'] =  [x['deputado_uriPartido'].values[-1] for x in info]
# graph.vs['Total Despesas (reais)'] = [get_total_expenses(x['deputado_id'].values[-1], start_date, end_date) for x in info]
for i, x in enumerate(info):
    for k, v in get_deputy_info(x['deputado_id'].values[-1]).items():
        if i == 0:
            graph.vs[k] = ['' for _ in range(len(graph.vs))]
        graph.vs[i][k] = v

graph.vs['betweenness'] = graph.betweenness()
graph.vs['closeness'] = graph.closeness()

summary(graph)

IGRAPH UNW- 561 31416 -- Câmara dos Deputados
+ attr: name (g), Foto (v), Partido (v), Partido URI (v), UF (v), URL (v), betweenness (v), cidade natal (v), closeness (v), community (v), community_leiden (v), e-mail (v), escolaridade (v), idade (v), name (v), sexo (v), situação (v), similarity (e), weight (e)


## Visualization

In [12]:
g = graphistry.bind(source='src', destination='dst',
                    point_label='name',
                    edge_weight='similarity',
                    edge_label='similarity',
                    edge_size='similarity',
                    point_color='community',
                    point_size='name')
(e_df, n_df) = g.igraph2pandas(graph)
g = g.nodes(n_df).edges(e_df)
g = g.nodes(lambda g: g._nodes.assign(community=g._nodes['community'].astype('int32')))
g = g.addStyle(bg={'color': 'white'}, page={'title': 'Câmara dos Deputados'})
g = g.encode_point_icon('Partido URI', categorical_mapping=party_logo, shape='circle')
g = g.settings(url_params={
    'pageTitle': 'Câmara dos Deputados',
    'play': 2000,
    'menu': True,
    'info': True,
    'strongGravity': True,
    'showArrows': False,
    'pointSize': 5.0,
    'pointsOfInterestMax': 100,
    'edgeInfluence': 1.0,
    'showLabels': True, 'showLabelOnHover': True,
    'showPointsOfInterest': True, 'showPointsOfInterestLabel': True, 'showLabelPropertiesOnHover': True})
g.plot(render=True)

<ipython-input-12-15c5726deac5>:8: DeprecationWarning: igraph2pandas deprecated; switch to from_igraph
  (e_df, n_df) = g.igraph2pandas(graph)
